# **Amazon Lookout for Equipment** - 익명화한 익스펜더 데이터셋에 대한 데모
*파트 2: 데이터셋 생성*

**버킷 이름을 적절하게 변경하세요.** 본 노트북은 Lookout for Equipment 서비스에 대해 S3 버킷 접근을 허용하는 역할을 갖고 있어야합니다. 본 단계에서 데이터 수집을 위해 해당 S3 위치에서 데이터를 읽을 수 있어야 합니다. 추론 스케줄링 단계에서 버킷에 데이터를 쓸 수 있어야 합니다.

**참고:** Amazon Lookout for Equipment에 대한 IAM 역할을 생성하지 않았다면 먼저 다음 [**IAM 역할 생성 가이드**](https://github.com/dast1/l4e_iam_role_configuration/blob/main/configure_IAM_role.md)를 따르십시오.

In [1]:
BUCKET = '<YOUR_BUCKET_NAME_HERE>'
PREFIX = 'data/training-data/expander/'

## 초기화
---
데이터 준비 노트북을 수행하면 저장소는 이제 다음과 같이 구성됩니다.
```
/lookout-equipment-demo
|
+-- data/
|   |
|   +-- labelled-data/
|   |   \-- labels.csv
|   |
|   \-- training-data/
|       \-- expander/
|           |-- subsystem-01
|           |   \-- subsystem-01.csv
|           |
|           |-- subsystem-02
|           |   \-- subsystem-02.csv
|           |
|           |-- ...
|           |
|           \-- subsystem-24
|               \-- subsystem-24.csv
|
+-- dataset/
|   |-- labels.csv
|   |-- tags_description.csv
|   |-- timeranges.txt
|   \-- timeseries.zip
|
+-- notebooks/
|   |-- 1_data_preparation.ipynb
|   |-- 2_dataset_creation.ipynb            <<< 본 노트북 <<<
|   |-- 3_model_training.ipynb
|   |-- 4_model_evaluation.ipynb
|   \-- 5_inference_scheduling.ipynb
|
+-- utils/
    |-- lookout_equipment_utils.py
    \-- lookoutequipment.json
```

### 임포트

In [2]:
%%sh
pip -q install --upgrade pip
pip -q install --upgrade awscli boto3 sagemaker
aws configure add-model --service-model file://../utils/lookoutequipment.json --service-name lookoutequipment

In [3]:
from IPython.core.display import HTML
HTML("<script>Jupyter.notebook.kernel.restart()</script>")

In [4]:
import boto3
import os
import pandas as pd
import pprint
import sagemaker
import sys
import time
import warnings

from datetime import datetime

# Lookout for Equipment API 호출 관리를 위한 Helper 함수
sys.path.append('../utils')
import lookout_equipment_utils as lookout

### 파라미터

In [5]:
warnings.filterwarnings('ignore')

DATA       = os.path.join('..', 'data')
LABEL_DATA = os.path.join(DATA, 'labelled-data')
TRAIN_DATA = os.path.join(DATA, 'training-data', 'expander')

ROLE_ARN = sagemaker.get_execution_role()
REGION_NAME = boto3.session.Session().region_name
DATASET_NAME = 'lookout-demo-training-dataset'

In [6]:
# 훈련 데이터 디렉토리의 List
# 디렉토리: 각 디렉토리는 하나의 하위 시스템에 대응됩니다.
components = []
for root, dirs, files in os.walk(f'{TRAIN_DATA}'):
    for subsystem in dirs:
        components.append(subsystem)

## 데이터셋 생성
---

### 데이터 스키마 생성하기

먼저 데이터셋의 스키마를 설정해야합니다. 아래 셀에서 `DATASET_COMPONENT_FIELDS_MAP`를 정의하십시오. `DATASET_COMPONENT_FIELDS_MAP`는 Python Dictionary (hashmap)입니다. Dictionary 각 항목의 키는 `Component` 이름이고 각 항목의 값은 열 이름의 List입니다. 열 이름은 csv 파일의 헤더와 정확히 일치해야합니다. 열 이름의 순서도 정확히 일치해야합니다. 예를 들어 여기서 사용할 예제의 데이터 스키마를 생성하려는 경우 Dictionary는 다음과 같습니다. 

```json
DATASET_COMPONENT_FIELDS_MAP = {
    "Component1": ['Timestamp', 'Tag1', 'Tag2',...],
    "Component2": ['Timestamp', 'Tag1', 'Tag2',...]
    ...
    "ComponentN": ['Timestamp', 'Tag1', 'Tag2',...]
}
```

구성 요소 이름이 S3의 폴더 명과 **정확히 일치**하는지 확인합니다 (**대소문자 구분**).
```json
DATASET_COMPONENT_FIELDS_MAP = {
    "subsystem-01": ['Timestamp', 'signal-026', 'signal-027',... , 'signal-092'],
    "subsystem-02": ['Timestamp', 'signal-022', 'signal-023',... , 'signal-096'],
    ...
    "subsystem-24": ['Timestamp', 'signal-083'],
}
```

In [7]:
DATASET_COMPONENT_FIELDS_MAP = dict()
for subsystem in components:
    subsystem_tags = ['Timestamp']
    for root, _, files in os.walk(f'{TRAIN_DATA}/{subsystem}'):
        for file in files:
            fname = os.path.join(root, file)
            current_subsystem_df = pd.read_csv(fname, nrows=1)
            subsystem_tags = subsystem_tags + current_subsystem_df.columns.tolist()[1:]

        DATASET_COMPONENT_FIELDS_MAP.update({subsystem: subsystem_tags})
        
        
lookout_dataset = lookout.LookoutEquipmentDataset(
    dataset_name=DATASET_NAME,
    component_fields_map=DATASET_COMPONENT_FIELDS_MAP,
    region_name=REGION_NAME,
    access_role_arn=ROLE_ARN
)

콘솔을 사용하는 경우 다음 문자열을 이용하여 **데이터셋 스키마**를 구성할 수 있습니다.

![dataset_schema](../assets/dataset-schema.png)

In [8]:
import pprint
pp = pprint.PrettyPrinter(depth=5)
pp.pprint(eval(lookout_dataset.dataset_schema))

{'Components': [{'Columns': [{'Name': 'Timestamp', 'Type': 'DATETIME'},
                             {'Name': 'signal-001', 'Type': 'DOUBLE'},
                             {'Name': 'signal-002', 'Type': 'DOUBLE'},
                             {'Name': 'signal-003', 'Type': 'DOUBLE'},
                             {'Name': 'signal-004', 'Type': 'DOUBLE'},
                             {'Name': 'signal-046', 'Type': 'DOUBLE'},
                             {'Name': 'signal-047', 'Type': 'DOUBLE'},
                             {'Name': 'signal-077', 'Type': 'DOUBLE'},
                             {'Name': 'signal-081', 'Type': 'DOUBLE'},
                             {'Name': 'signal-106', 'Type': 'DOUBLE'},
                             {'Name': 'signal-107', 'Type': 'DOUBLE'}],
                 'ComponentName': 'subsystem-05'},
                {'Columns': [{'Name': 'Timestamp', 'Type': 'DATETIME'},
                             {'Name': 'signal-022', 'Type': 'DOUBLE'},
                       

### 데이터셋 생성하기

In [9]:
lookout_dataset.create()

Dataset "lookout-demo-training-dataset" already exists and can be used to ingest data or train a model.


데이터셋이 생성되었지만 현재 비어 있으며 이전 노트북에서 준비한 S3 위치에서 시계열 데이터를 수신, 수집할 준비가 되었습니다.

![dataset_schema](../assets/dataset-created.png)

## 데이터셋으로 데이터 수집하기
---
생성한 Lookout for Equipment 데이터셋으로 데이터 일부를 수집하는데 사용할 매개 변수 값 전부를 재확인해보겠습니다. 

In [10]:
# ROLE_ARN, BUCKET, PREFIX, DATASET_NAME

Lookout for Equipment 데이터셋에서 수집 작업을 시작합니다.

In [11]:
response = lookout_dataset.ingest_data(BUCKET, PREFIX)

수집이 시작됩니다. 현재 데이터 크기 (약 1.5GB)에서 5-10분 정도 걸립니다.

![dataset_schema](../assets/dataset-ingestion-in-progress.png)

In [12]:
# 수집 작업 ID와 상태를 가져옵니다.
data_ingestion_job_id = response['JobId']
data_ingestion_status = response['Status']

# 수집이 완료될 때까지 기다립니다.
print("=====Polling Data Ingestion Status=====\n")
lookout_client = lookout.get_client(region_name=REGION_NAME)
print(str(pd.to_datetime(datetime.now()))[:19], "| ", data_ingestion_status)

while data_ingestion_status == 'IN_PROGRESS':
    time.sleep(60)
    describe_data_ingestion_job_response = lookout_client.describe_data_ingestion_job(JobId=data_ingestion_job_id)
    data_ingestion_status = describe_data_ingestion_job_response['Status']
    print(str(pd.to_datetime(datetime.now()))[:19], "| ", data_ingestion_status)
    
print("\n=====End of Polling Data Ingestion Status=====")

=====Polling Data Ingestion Status=====

2021-04-02 20:07:12 |  IN_PROGRESS
2021-04-02 20:08:13 |  IN_PROGRESS
2021-04-02 20:09:14 |  IN_PROGRESS
2021-04-02 20:10:14 |  IN_PROGRESS
2021-04-02 20:11:14 |  IN_PROGRESS
2021-04-02 20:12:14 |  IN_PROGRESS
2021-04-02 20:13:15 |  SUCCESS

=====End of Polling Data Ingestion Status=====


콘솔에서 보듯이 이제 수집이 완료되어야합니다.

![dataset_schema](../assets/dataset-ingestion-done.png)

## 결론
---

이 노트북에서는 **Lookout for Equipment 데이터셋**을 생성하고 해당 데이터셋을 이용하여 이전에 업로드한 S3 데이터를 수집했습니다. **본 데이터를 기반으로 모델을 학습하려면 다음 노트북으로 바로 이동하세요.**

In [13]:
# 본 시리즈의 다음 노트북에서 재사용하게끔 데이터셋 명을 저장해둬야 합니다. 
dataset_fname = os.path.join(DATA, 'dataset_name.txt')
with open(dataset_fname, 'w') as f:
    f.write(DATASET_NAME)